In [1]:
from gym_match3.envs import Match3Env
import gym
import matplotlib.pyplot as plt

from matplotlib import style
import numpy as np

env = Match3Env()
#obs, reward, done, info = env.step(0) 
import torch
import cv2

from PIL import Image

from gym_match3.envs.levels import LEVELS #  default levels
from gym_match3.envs.levels import Match3Levels, Level


from gym_match3.envs.game import (Board,
                                  RandomBoard,
                                  CustomBoard,
                                  Point,
                                  Cell,
                                  AbstractSearcher,
                                  MatchesSearcher)


from gym_match3.envs.game import OutOfBoardError, ImmovableShapeError

In [2]:
def one_hot_encoding(obs):
    
    obs = obs.reshape(obs.shape[1],obs.shape[2]).astype(int)
    
    grid_onehot = np.zeros(shape=(env.n_shapes, obs.shape[0], obs.shape[1]))
    table = {i:i for i in range(0,10)}
    
    for i in range(obs.shape[0]):
        for j in range(obs.shape[1]):
            grid_element = obs[i][j]
            grid_onehot[table[grid_element]][i][j]=1
            
    return grid_onehot



def plot_obs(observation=None):
    
    d = {0: (255, 255, 255),   #白色
     1: (0, 255, 255),      #黃色
     2: (0, 0, 255) ,      #紅色
     3: (230,224,176) ,   #灰藍色 
     4: (0,0,0), #黑色  
     5: (100,100,100),
        6:(150,150,150),
        7:(200,200,200),
        8:(200,150,250),
        9:(250,200,150),
        10:(150,200,250)}              
    
    
    background = np.zeros((observation.shape[1],observation.shape[2],3), dtype=np.uint8) #黑色的背景

    for color in range(env.n_shapes):
        result = np.where(one_hot_encoding(observation)[color] == 1)
        
        listOfCoordinates= list(zip(result[0], result[1]))
        
        for cord in listOfCoordinates:
            background[cord] = d[color]     
            
    img = Image.fromarray(background, 'RGB')
     
    
    #img = img.resize((500, 800)) 
    cv2.imshow("image", np.array(img))
    
    cv2.waitKey(200)
    
    
    
def Getlevels(WidthAndHeight,shapes):
    LEVELS = [Level(WidthAndHeight,WidthAndHeight,shapes, np.zeros((WidthAndHeight,WidthAndHeight)).tolist())]
    return LEVELS

env = Match3Env(levels=Match3Levels(Getlevels(4,10)))

In [3]:
from gym_match3.envs import Match3Env
import gym
from PIL import Image
import cv2
import numpy as np
from random import choice


env = Match3Env(levels=Match3Levels(Getlevels(4,10)))

for i_episode in range(1): #玩一次遊戲
     
    observation = env.reset() 
    
    while True:
    
        
        action = choice(env.get_validate_actions())

         
        observation, reward, done, info = env.step(action)
        env.game.get_possible_moves()
        

        print('the swap of coordinate is: ',list(env.get_available_actions()[action]))

        print(observation)

        if done:           
            break
env.close()

TypeError: list indices must be integers or slices, not frozenset